# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 6 2023 11:32AM,256710,10,Eme-108513,Emerginnova,Released
1,Feb 6 2023 11:32AM,256710,10,Enova-11317,Emerginnova,Released
2,Feb 6 2023 11:31AM,256709,10,WT5578 Out,"Snap Medical Industries, LLC",Released
3,Feb 6 2023 11:27AM,256707,12,U1209,"Uniderm USA, Inc",Released
4,Feb 6 2023 11:20AM,256705,16,SSF_C2W6,Sartorius Stedim Filters Inc.,Released
5,Feb 6 2023 11:11AM,256704,10,9123603,"Snap Medical Industries, LLC",Released
6,Feb 6 2023 11:11AM,256704,10,9123605,"Snap Medical Industries, LLC",Released
7,Feb 6 2023 11:11AM,256704,10,9123606,"Snap Medical Industries, LLC",Released
8,Feb 6 2023 11:11AM,256704,10,9123607,"Snap Medical Industries, LLC",Released
9,Feb 6 2023 11:11AM,256704,10,9123608,"Snap Medical Industries, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,256704,Released,7
25,256705,Released,1
26,256707,Released,1
27,256709,Released,1
28,256710,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256704,NaN,NaN,7.0
256705,NaN,NaN,1.0
256707,NaN,NaN,1.0
256709,NaN,NaN,1.0
256710,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256656,1.0,1.0,1.0
256671,0.0,0.0,48.0
256673,0.0,0.0,36.0
256676,0.0,0.0,2.0
256681,0.0,4.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256656,1,1,1
256671,0,0,48
256673,0,0,36
256676,0,0,2
256681,0,4,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256656,1,1,1
1,256671,0,0,48
2,256673,0,0,36
3,256676,0,0,2
4,256681,0,4,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256656,1,1,1
1,256671,,,48
2,256673,,,36
3,256676,,,2
4,256681,,4,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 6 2023 11:32AM,256710,10,Emerginnova
2,Feb 6 2023 11:31AM,256709,10,"Snap Medical Industries, LLC"
3,Feb 6 2023 11:27AM,256707,12,"Uniderm USA, Inc"
4,Feb 6 2023 11:20AM,256705,16,Sartorius Stedim Filters Inc.
5,Feb 6 2023 11:11AM,256704,10,"Snap Medical Industries, LLC"
12,Feb 6 2023 11:03AM,256703,12,"NBTY Global, Inc."
13,Feb 6 2023 11:01AM,256697,20,"Exact-Rx, Inc."
14,Feb 6 2023 10:54AM,256698,19,"Methapharm, Inc."
15,Feb 6 2023 10:50AM,256696,12,"NBTY Global, Inc."
16,Feb 6 2023 10:40AM,256695,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 6 2023 11:32AM,256710,10,Emerginnova,,,2
1,Feb 6 2023 11:31AM,256709,10,"Snap Medical Industries, LLC",,,1
2,Feb 6 2023 11:27AM,256707,12,"Uniderm USA, Inc",,,1
3,Feb 6 2023 11:20AM,256705,16,Sartorius Stedim Filters Inc.,,,1
4,Feb 6 2023 11:11AM,256704,10,"Snap Medical Industries, LLC",,,7
5,Feb 6 2023 11:03AM,256703,12,"NBTY Global, Inc.",,,1
6,Feb 6 2023 11:01AM,256697,20,"Exact-Rx, Inc.",,,1
7,Feb 6 2023 10:54AM,256698,19,"Methapharm, Inc.",,,1
8,Feb 6 2023 10:50AM,256696,12,"NBTY Global, Inc.",,,1
9,Feb 6 2023 10:40AM,256695,15,"Brookfield Pharmaceuticals, LLC",,32,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 11:32AM,256710,10,Emerginnova,2,,
1,Feb 6 2023 11:31AM,256709,10,"Snap Medical Industries, LLC",1,,
2,Feb 6 2023 11:27AM,256707,12,"Uniderm USA, Inc",1,,
3,Feb 6 2023 11:20AM,256705,16,Sartorius Stedim Filters Inc.,1,,
4,Feb 6 2023 11:11AM,256704,10,"Snap Medical Industries, LLC",7,,
5,Feb 6 2023 11:03AM,256703,12,"NBTY Global, Inc.",1,,
6,Feb 6 2023 11:01AM,256697,20,"Exact-Rx, Inc.",1,,
7,Feb 6 2023 10:54AM,256698,19,"Methapharm, Inc.",1,,
8,Feb 6 2023 10:50AM,256696,12,"NBTY Global, Inc.",1,,
9,Feb 6 2023 10:40AM,256695,15,"Brookfield Pharmaceuticals, LLC",2,32,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 11:32AM,256710,10,Emerginnova,2,,
1,Feb 6 2023 11:31AM,256709,10,"Snap Medical Industries, LLC",1,,
2,Feb 6 2023 11:27AM,256707,12,"Uniderm USA, Inc",1,,
3,Feb 6 2023 11:20AM,256705,16,Sartorius Stedim Filters Inc.,1,,
4,Feb 6 2023 11:11AM,256704,10,"Snap Medical Industries, LLC",7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 11:32AM,256710,10,Emerginnova,2.0,NaN,NaN
1,Feb 6 2023 11:31AM,256709,10,"Snap Medical Industries, LLC",1.0,NaN,NaN
2,Feb 6 2023 11:27AM,256707,12,"Uniderm USA, Inc",1.0,NaN,NaN
3,Feb 6 2023 11:20AM,256705,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
4,Feb 6 2023 11:11AM,256704,10,"Snap Medical Industries, LLC",7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 11:32AM,256710,10,Emerginnova,2.0,0.0,0.0
1,Feb 6 2023 11:31AM,256709,10,"Snap Medical Industries, LLC",1.0,0.0,0.0
2,Feb 6 2023 11:27AM,256707,12,"Uniderm USA, Inc",1.0,0.0,0.0
3,Feb 6 2023 11:20AM,256705,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
4,Feb 6 2023 11:11AM,256704,10,"Snap Medical Industries, LLC",7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2310220,105.0,9.0,0.0
12,770106,3.0,0.0,0.0
15,256695,2.0,32.0,0.0
16,770072,2.0,5.0,0.0
19,770030,4.0,1.0,1.0
20,256697,1.0,0.0,0.0
21,770052,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2310220,105.0,9.0,0.0
1,12,770106,3.0,0.0,0.0
2,15,256695,2.0,32.0,0.0
3,16,770072,2.0,5.0,0.0
4,19,770030,4.0,1.0,1.0
5,20,256697,1.0,0.0,0.0
6,21,770052,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,105.0,9.0,0.0
1,12,3.0,0.0,0.0
2,15,2.0,32.0,0.0
3,16,2.0,5.0,0.0
4,19,4.0,1.0,1.0
5,20,1.0,0.0,0.0
6,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,105.0
1,12,Released,3.0
2,15,Released,2.0
3,16,Released,2.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Executing,9.0,0.0,32.0,5.0,1.0,0.0,0.0
Released,105.0,3.0,2.0,2.0,4.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,9.0,0.0,32.0,5.0,1.0,0.0,0.0
2,Released,105.0,3.0,2.0,2.0,4.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,9.0,0.0,32.0,5.0,1.0,0.0,0.0
2,Released,105.0,3.0,2.0,2.0,4.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()